---
## 任意の単語数で分割する関数

In [ ]:
import os
import re
import gc
import json
import glob
import pickle
import string
import random
import itertools
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

import nltk
from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import CometLogger
import transformers
from transformers import BertModel, BertForTokenClassification

%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 300)
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
def load_data(data_dir):    
    # Testデータの読み込み
    test_files = glob.glob(data_dir + "test/*.json")

    test = pd.DataFrame()

    # jsonからDataFrameに
    for tar in test_files:
        file_data = pd.read_json(tar)
        file_data.insert(0,'pub_id', tar.split('/')[-1].split('.')[0])
        test = pd.concat([test, file_data])
    
    return test


def preprocess_text(text: str) -> str:
    """
    テキストの前処理　クリーニング
    """
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    
    return text

In [ ]:
data_dir = '../input/coleridgeinitiative-show-us-the-data/'
test = load_data(data_dir)

test['text'] = test['text'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else x)

In [ ]:
test.head()

## Expand Dataset

In [ ]:
# 確認のため一つのpub_idに絞る
test_ids = test['pub_id'].unique()
tar = test[test['pub_id'] == test_ids[0]]

In [ ]:
tar

In [ ]:
# それぞれのセクションのテキストの数を計算する
tar['text_len'] = tar['text'].apply(lambda x: len(x.split(' ')) if isinstance(x, str) else 0)
tar[['pub_id', 'section_title', 'text_len']]

In [ ]:
tar.drop('text_len', axis=1, inplace=True)

In [ ]:
# 指定したmax_lenより大きい場合は分割して行を分ける
# 重複も許すように設計する

# テキストの数
max_len = 32
# 重複する単語数
override = 10
# 結果格納用データフレーム
res = pd.DataFrame()

for i in range(len(tar)):
    row = tar.iloc[i]
    tar_text = row['text'].split(' ')
    text_len = len(tar_text)
    
    # 単語数がmax_lenより小さい場合はそのまま
    if text_len < max_len:
        res = pd.concat([res, pd.DataFrame(row).T], axis=0)
        continue
    
    # 単語数がmax_lenより大きい場合は分割する
    elif text_len > max_len:
        # 分割する数を計算する
        num_divide = int(np.ceil(text_len / (max_len - override)))
        # 分割する分行を複製しておく（データフレーム化）
        tmp_df = pd.DataFrame([row] * num_divide)
        # 分割後のテキストを格納しておくリスト
        divided_texts = []
        
        # max_lenごとのテキストに分割する
        for i in range(len(tmp_df)):
            div_text = tar_text[int(i * (max_len - override)) : int(i * (max_len - override) + max_len)]
            # リストから文字列に直す
            div_text = ' '.join(div_text)
            # 結果を一旦リストにまとめておく
            divided_texts.append(div_text)
            
        # 複製しておいたデータフレームに置換
        tmp_df['text'] = divided_texts
        # 全体のデータフレームに結合
        res = pd.concat([res, tmp_df], axis=0)
        
    # 動作確認のため強制終了
    break

In [ ]:
res

In [ ]:
# 単語数の確認
res['text_len'] = res['text'].apply(lambda x: len(x.split(' ')))
res

In [ ]:
# 対象のテキストを一応表示しておく
tar['text'].values[0]

## 関数化

上の処理を関数化しておく

In [ ]:
def preprocess_text(text: str) -> str:
    """
    テキストの前処理　クリーニング
    """
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    
    return text


def expand_data(df, max_len, override=0) -> pd.DataFrame:
    """
    指定したmax_lenを超えるテキストに対して分割を行う関数
    
    ---------------------------------------
    Parameters
    
    df: pd.DataFrame
        拡張対象のデータフレーム
        pub_id, section_title, textが存在していること
    max_len: int
        分割する単語数
    override: int
        分割する際に重複する単語数
        
    ---------------------------------------
    Returns
    
    res: pd.DataFrame
        分割したテキストで構成されたデータフレーム
    
    """
    # 結果格納用データフレーム
    res = pd.DataFrame()
    
    # テキストの前処理
    df['text_clean'] = df['text'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else x)
    
    ids = df['pub_id'].unique()
    
    for _id in ids:   
        tar = df[df['pub_id'] == _id]

        for i in range(len(tar)):
            row = tar.iloc[i]
            tar_text = row['text_clean'].split(' ')
            text_len = len(tar_text)

            # 単語数がmax_lenより小さい場合はそのまま
            if text_len <= max_len:
                res = pd.concat([res, pd.DataFrame(row).T], axis=0)   # Version 2で修正

            # 単語数がmax_lenより大きい場合は分割する
            elif text_len > max_len:
                # 分割する数を計算する
                num_divide = int(np.ceil(text_len / (max_len - override)))
                # 分割する分行を複製しておく（データフレーム化）
                tmp_df = pd.DataFrame([row] * num_divide)
                # 分割後のテキストを格納しておくリスト
                divided_texts = []

                # max_lenごとのテキストに分割する
                for i in range(len(tmp_df)):
                    div_text = tar_text[int(i * (max_len - override)) : int(i * (max_len - override) + max_len)]
                    # リストから文字列に直す
                    div_text = ' '.join(div_text)
                    # 結果を一旦リストにまとめておく
                    divided_texts.append(div_text)

                # 複製しておいたデータフレームに置換
                tmp_df['text_clean'] = divided_texts
                # 全体のデータフレームに結合
                res = pd.concat([res, tmp_df], axis=0)
                
    # 余計な行を削除する
    res = res.dropna()
    res = res.reset_index(drop=True)
    
    return res

In [ ]:
%%time
res = expand_data(test, max_len=128, override=3)

In [ ]:
res

In [ ]:
print('拡張前')
print(test.shape)
print('拡張後')
print(res.shape)

In [ ]:
data_dir = '../input/coleridgeinitiative-show-us-the-data/'
test = load_data(data_dir)

res = expand_data(test, max_len=512, override=10)

In [ ]:
test[test['pub_id'] == '8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60']

In [ ]:
res[res['pub_id'] == '8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60']